<a href="https://colab.research.google.com/github/ferdouszislam/pytorch-practice/blob/main/cnn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [16]:
# hyper parameters
num_epochs = 20
batch_size = 64
learning_rate = 0.001

In [4]:
# load dataset
transform = transforms.Compose(
    [
     transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                             download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck') 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
# convolutional neural network
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()

    # 5x5 conv kernel/filter
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5) 
    
    # 2x2 maxpooling
    self.pool =  nn.MaxPool2d(kernel_size=2, stride=2) 
    
    # 5x5 conv kernel/filter
    self.conv2 = nn.Conv2d(6, 16, 5)
    
    # 3x32x32 --conv1--> 6x28x28 --maxpool--> 6x14x14
    # 6x14x14 --conv2--> 16x10x10 --maxpool--> 16x5x5
    self.fc1 = nn.Linear(16*5*5, 120) 
    self.fc2 = nn.Linear(120, 84) 
    self.fc3 = nn.Linear(84, 10)
    self.relu = nn.ReLU()

  def forward(self, input):
    # conv1 -> relu -> maxpool
    output = self.pool(self.relu(self.conv1(input))) 
    # conv2 -> relu -> maxpool
    output = self.pool(self.relu(self.conv2(output)))

    # flatten
    output = output.reshape(-1, 16*5*5)
    # -> fc1 -> relu
    output = self.relu(self.fc1(output))
    # -> fc2 -> relu
    output = self.relu(self.fc2(output))
    # fc3 (no activation at end)
    output = self.fc3(output)

    return output

In [11]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [19]:
for epoch in range(num_epochs):
  avg_batch_loss = 0
  for step, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # accumulate batch loss
    avg_batch_loss+=loss

  avg_batch_loss /= len(train_loader)

  print(f'epoch {epoch+1}/{num_epochs}, loss = {avg_batch_loss:.4f}')

epoch 1/20, loss = 1.2382
epoch 2/20, loss = 1.2164
epoch 3/20, loss = 1.1937
epoch 4/20, loss = 1.1727
epoch 5/20, loss = 1.1545
epoch 6/20, loss = 1.1355
epoch 7/20, loss = 1.1186
epoch 8/20, loss = 1.1008
epoch 9/20, loss = 1.0837
epoch 10/20, loss = 1.0700
epoch 11/20, loss = 1.0550
epoch 12/20, loss = 1.0403
epoch 13/20, loss = 1.0263
epoch 14/20, loss = 1.0144
epoch 15/20, loss = 1.0011
epoch 16/20, loss = 0.9878
epoch 17/20, loss = 0.9759
epoch 18/20, loss = 0.9642
epoch 19/20, loss = 0.9514
epoch 20/20, loss = 0.9402


In [21]:
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      # max returns (value ,index)
      _, predicted = torch.max(outputs, 1)
      n_samples += labels.size(0)
      n_correct += (predicted == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc}%')

Accuracy of the network: 62.47%
